In [ ]:
import os, re, regex, random, nltk
import pandas as pd
import numpy as np
import emoji
import plotly.express as px
from collections import Counter
import matplotlib.pyplot as plt
import datetime
from wordcloud import WordCloud, STOPWORDS, ImageColorGenerator
%matplotlib inline

In [ ]:
def date_time_ios(s):
    pattern = r"^\[([0-9]+)(\/)([0-9]+)(\/)([0-9][0-9]) ([0-9]+):([0-9]+):([0-9][0-9])\]"
    result = re.match(pattern, s)
    if result:
        return True
    return False

def get_contact(s):
    patterns = [
        "([\w]+):",
        "([\w]+[\s]+[\w]+):",
        "([\w]+[\s]+[\w]+[\s]+[\w]+):",
        "([+]\d{2} \d{3} \d{3} \d{4}):",
    ]
    pattern = '^' + '|'.join(patterns)
    result = re.match(pattern, s)
    if result:
        return True
    return False

def emojis(s):
    emoji_list = []
    data = re.findall('\W', s)
    for word in data:
        if any(char in emoji.UNICODE_EMOJI for char in word):
            emoji_list.append(word)
    return emoji_list

def get_message(line):
    splitline = line.splitlineplit("] ")
    dateTime = splitline[0]
    date, time = dateTime.split(" ")
    date = date[1:]
    message = " ".join(splitline[1:])
    if get_contact(message):
        splitMessage = message.split(": ")
        contact = splitMessage[0]
        message = " ".join(splitMessage[1:])
    else:
        contact = None
    return date, time, contact, message

In [ ]:
converstion = "WhatsApp_Group_Chats_PDTP3.txt"
data = []
with open(converstion, encoding="utf-8") as fp:
    fp.readline()
    message_buffer = []
    date, time, contact = None, None, None
    while True:
        line = fp.readline()
        if not line:
            break
        line = line.strip()
        line = re.sub(r'[\u202a\u202c]', ' ', line)
        line = re.sub(r'[\xa0]', ' ', line)
        if date_time_ios(line):
            if len(message_buffer) > 0:
                data.append([date, time, contact, ' '.join(message_buffer)])
            message_buffer.clear()
            date, time, contact, message = get_message(line)
            message_buffer.append(message)
        else: 
            message_buffer.append(line)

df = pd.DataFrame(data, columns=['Date', 'Time', 'Contact', 'Message'])
df["Date"] = pd.to_datetime(df["Date"])
df["Message"] = df["Message"].str.lower()
df["Contact"] = df["Contact"].str.lower()
df.Date = pd.to_datetime(df.Date)
df = df.dropna()
df.reset_index(inplace= True)

In [ ]:
df.head(20)